<a href="https://colab.research.google.com/github/abhikadam27/Deep-Learning/blob/main/Transfer_learning_Cat_vs_Dog_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cats v/s Dogs Prediction

In [ ]:
cd /content/drive/MyDrive

## Loading zip file

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/dogs-vs-cats.zip','r') as zipObj:
  #Extract all the contents of zip file in current directory
  zipObj.extractall()

## Extracting train and test folders

In [ ]:
with ZipFile('train.zip','r') as zipObj:
    zipObj.extractall()
with ZipFile('test1.zip','r') as zipObj:
    zipObj.extractall()

## Create sub-directories so the data is in this format

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from os import makedirs
from os import listdir
from shutil import copyfile

In [ ]:
#create directories
dataset_home = 'dataset_dogs_vs_cat/'
subdirs = ['train/','test/']
for subdir in subdirs:
    #create label subdirectories
    labeldirs = ['dogs/', 'cats/']
    for labeldir in labeldirs:
      newdir = dataset_home + subdir + labeldir
      makedirs(newdir, exist_ok = True)

## Now change the dataset to fit this format

In [ ]:
#create directories
from random import seed, random
seed(1)
#define ratio of pictures to use for validation
val_ratio = 0.25
#copy training dataset images into subdirectories
src_directory = '/content/train'
dataset_home = '/content/dataset_dogs_vs_cat/'
for file in listdir(src_directory):
    src = src_directory +'/'+file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('cat'):
        dst = dataset_home+dst_dir+'cats/'+file
        copyfile(src,dst)
    elif file.startswith('dog'):
        dst = dataset_home+dst_dir+'dogs/'+ file
        copyfile(src,dst)

In [ ]:
Directory = '/content/dataset_dogs_vs_cat/train'
categories = [['cats'],['dogs']]

## Visualization

In [ ]:
for category in categories[1]:
  folder = os.path.join(Directory, category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder, img)
    img_arr = cv2.imread(img_path)
    plt.imshow(img_arr)
    break

In [ ]:
for category in categories[0]:
  folder = os.path.join(Directory, category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder, img)
    img_arr = cv2.imread(img_path)
    plt.imshow(img_arr)
    break

## Model Building

In [ ]:
import keras
#import tensorflow as tf 
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
model = VGG16(include_top=False, input_shape=(224,224,3))

In [ ]:
#mark loaded layers as not trainable
for layer in model.layers:
  layer.trainable = False
#add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)

#Define new model
model = Model(inputs = model.inputs, outputs = output)

#COmpile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Read the model using generator
datagen = ImageDataGenerator(featurewise_center=True)
#specify imagenet mean values for centering
datagen.mean = [128.68, 116.779, 103.939]

#prepare iterator
train_it = datagen.flow_from_directory('dataset_dogs_vs_cat/train/', class_mode = 'binary', batch_size = 64, target_size = (224,224))
test_it = datagen.flow_from_directory('dataset_dogs_vs_cat/test/', class_mode = 'binary', batch_size = 64, target_size = (224,224))

In [ ]:
#fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=5, verbose=1)

## Plot losses and accuracy

In [ ]:
import matplotlib.pyplot as plt
import sys
plt.subplot(211)
plt.title("Cross Entropy Loss")
plt.plot(history.history['loss'], color = 'blue', label = 'train')
plt.plot(history.history['val_loss'], color = 'orange', label='test')
plt.legend()

#save the plot file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename+'plot.png')

In [ ]:
#plot accuracy
#plt.subplots(212)
plt.title("Classification Accuracy")
plt.plot(history.history['accuracy'], color = 'blue', label = 'train')
plt.plot(history.history['val_accuracy'], color = 'orange', label='test')
plt.legend()

#save the plot file
#filename = sys.argv[0].split('/')[-1]
#plt.savefig(filename+'plot1.png')

## Model Saving

In [ ]:
#save the model for future use
model.save('final_model.h5')

## Predicting the outputs

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
#load the model
model = load_model('/content/drive/MyDrive/final_model.h5')
#load & prepare image
img = load_img('/content/drive/MyDrive/train/cat.10057.jpg', target_size = (224,224))



In [ ]:
img

We can see the image is of a cat that means our model must predict it as cat

In [ ]:
#convert to array
img = img_to_array(img)
#reshape into a single sample with 3 channels
img = img.reshape(1,224,224,3)

#center pixel data
img = img.astype('float32')
img = img - [123.68, 116.779, 103.939]

# 1 is for dogs and 0 is for cats
result = model.predict(img)
print(result)

**Great the predicted value is near about zero(0) which means our model predicted the image accurately.**